#GPT NEO based discord chatbot

## Discord API
You just need to insert your discord API and then run all the cells

In [ ]:
discord_token = '' #@param {type:"string"}

##Module installation
this will install all the necessary modules

In [ ]:
!pip install transformers
!pip install discord.py
!pip install nest_asyncio

##Download and load GPT NEO model.
It will take a little bit

In [ ]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

model = GPTNeoForCausalLM.from_pretrained('EleutherAI/gpt-neo-1.3B')
tokenizer = GPT2Tokenizer.from_pretrained('EleutherAI/gpt-neo-1.3B')

# add padding token to tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# set padding token id to the id of the padding token
model.config.pad_token_id = tokenizer.pad_token_id
model.cuda()

##Mode setup
You can use this cell to select or create the desired mode
To create a mode just follow this example:


```python
modes['mode_name'] = { 'prompt' : 'insert text that is put at the beginning of the input',

message'
                      }
```



In [ ]:
modes = dict()

modes['chat'] = {'prompt': 'Self Generated AI Humanoid',
            }

mode = modes['chat']


##Function to get GPT's answer from discord chat history

You can play with time_limit and max_length using the sliders. Time limit is very useful if you want fast replies by GPT

In [ ]:
time_limit = 4 #@param {type:"slider", min:1, max:100, step:1}
max_length = 813 #@param {type:"slider", min:100, max:5000, step:1}

def AI_answer(string):
  in_string = mode['prompt'] + string
  inputs = tokenizer.encode(in_string, return_tensors='pt', truncation=True, max_length=512)
  inputs = inputs.cuda()
  attention_mask = inputs.ne(tokenizer.pad_token_id).float()
  outputs = model.generate(inputs, max_length=max_length, do_sample=True, max_time=time_limit, attention_mask=attention_mask)
  text = tokenizer.decode(outputs[0], skip_special_tokens=True)

  stripped_text = text[len(in_string):]

  #preprocessing answer
  return stripped_text


##Discord bot

In [ ]:
import discord
import asyncio
import nest_asyncio
nest_asyncio.apply()

client = discord.Client(intents=discord.Intents.default())

@client.event
async def on_ready():
    print('We have logged in as {0.user}'.format(client))

@client.event
async def on_message(message):
    if message.author == client.user:
        return

    messages = []
    async for m in message.channel.history():
        messages.append(m)

    #create input string
    in_string = ''
    for message in reversed(messages):
      
      if message.author == client.user:

        output = AI_answer(in_string)

    await message.channel.send(output)

client.run(discord_token)